In [1]:
import numpy as np
import sys
from pathlib import Path

# Make `import src.*` work when running from nested directory
REPO_ROOT = Path.cwd().resolve()
if not (REPO_ROOT / "src").exists() and (REPO_ROOT.parent / "src").exists():
    REPO_ROOT = REPO_ROOT.parent.resolve()

if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from src import (
    ExperimentConfig,
    get_dataset,
    load_config,
    list_datasets,
    list_models,
    run_pipeline,
)

# Load a base YAML config from ./config/ (convenience wrapper)
cfg = load_config("pgd_eps0p05_pca5")

print("datasets:", list_datasets())
print("models:", list_models())

datasets: ['breast_cancer_tabular', 'cifar10', 'cifar100', 'fashion_mnist', 'mnist', 'synthetic_shapes_2class', 'synthetic_shapes_3class', 'two_moons']
models: ['minicnn', 'two_moons_mlp']


In [2]:
# Load MNIST (requires torchvision). If DOWNLOAD=False and files are missing, set DOWNLOAD=True or place MNIST under DATA_ROOT.
try:
    bundle_full = get_dataset("mnist", cfg, overrides={"DOWNLOAD": True})
except Exception as e:
    raise RuntimeError(
        "Failed to load MNIST. If you do not have MNIST locally, set DOWNLOAD=True, or point DATA_ROOT at an existing torchvision dataset directory.\n"
        f"Original error: {e!r}"
    )

print("Full MNIST:")
print("  X_train:", bundle_full.X_train.shape, "y_train:", bundle_full.y_train.shape)
print("  X_val:  ", bundle_full.X_val.shape, "y_val:  ", bundle_full.y_val.shape)
print("  X_test: ", bundle_full.X_test.shape, "y_test: ", bundle_full.y_test.shape)
print("  meta:", bundle_full.meta)

Full MNIST:
  X_train: (45000, 1, 28, 28) y_train: (45000,)
  X_val:   (15000, 1, 28, 28) y_val:   (15000,)
  X_test:  (10000, 1, 28, 28) y_test:  (10000,)
  meta: {'input_kind': 'image', 'clip': (0.0, 1.0), 'num_classes': 10, 'channels': 1, 'source': 'torchvision', 'root': './data', 'download': True}


In [ ]:
# End-to-end run on MNIST using MiniCNN
# NOTE: MNIST is grayscale, so we must set in_channels=1 for MiniCNN.
result = run_pipeline(
    dataset_name="mnist",
    model_name="minicnn",
    cfg=cfg,
    dataset_overrides={"DOWNLOAD": True},
    model_kwargs={"in_channels": 1},
    max_points_for_scoring=400,  # optional: keeps scoring runtime manageable
    seed=cfg.seed,
)

print("AUROC:", result.eval.metrics["roc_auc"])
print("AUPRC:", result.eval.metrics["pr_auc"])
print("FPR@95TPR:", result.eval.metrics["fpr_at_tpr95"])

Epoch [10/20] Train Loss: 0.0337, Train Acc: 98.93%, Val Loss: 0.0405, Val Acc: 98.75%
Epoch [20/20] Train Loss: 0.0100, Train Acc: 99.67%, Val Loss: 0.0406, Val Acc: 99.09%


Tried running this but my macbook is to slow...
